In [10]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, auc, precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint

# import custom modules
import sys
sys.path.insert(1, '../utils/')
import model_utils

In [11]:
df = pd.read_csv("../../pre_processing/datasets/boruta_filtered.csv", sep = ",")

In [12]:
if "RA" in df.columns:
    print("Removing column RA")
    df.drop('RA', inplace=True, axis=1)

In [13]:
df.head()

,EVADIDO,GRADE_CORRENTE,NOTA_MEDIA,TOT_MAT_CURSADAS,TOT_APROVACOES,TOT_REPROVACOES,TOT_REPROV_FREQ,TOT_REPROV_NOTA_FREQ,NCS005,NCS012,...,SIF006,SIF029,SIF030,SIF038,SIF039,SIF070,NCS037,NCS040,SIF072,SIF073
0,0,6,7.0,10,9,1,0,1,0,0,...,0,4,0,0,0,0,0,0,1,1
1,1,5,5.5,18,15,3,3,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,1,4,3.0,12,6,6,0,6,0,1,...,4,0,0,0,0,0,0,0,0,0
3,1,4,5.1,49,37,12,10,2,0,1,...,1,1,1,5,4,1,1,1,0,0
4,0,6,5.4,10,8,2,2,0,0,0,...,0,0,0,0,0,0,0,0,2,1


In [14]:
# Run all predictions based on fold n_splits
def run_prediction(X_train, X_test, y_train, y_test):
    
    # used to evaluate best model params
    gridsearch_metrics = {
            'REC':'recall', 
            'AUC':'roc_auc', 
            'Accuracy': 'accuracy'
    }

    # model gridsearch params
    model_params = {
        'weights': ['uniform', 'distance'], 
        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'p': [1, 2], # 1 = minkowski and 2 = euclidian
        'n_jobs': [15],
        'algorithm': ["auto"],
        "leaf_size": [10, 20, 30, 40, 50]
    }
    
    # gridsearch initialization
    clf = GridSearchCV(KNeighborsClassifier(),
                       param_grid = model_params,
                       scoring = gridsearch_metrics,
                       refit='Accuracy'
                      )
    
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    best_k = clf.best_params_["n_neighbors"]
    predict_data = {
        "model_algorithm": "knn",
        "epoch_best_k": best_k,
        "confusion_matrix": confusion_matrix(y_test, predictions),
        "classification_report": classification_report(y_test, predictions),
        "model_accuracy": accuracy_score(y_test, predictions),
        "epoch_params": clf.best_params_
    }

    return predict_data

In [17]:
def main():
    X = df.iloc[:, df.columns != "EVADIDO"].values
    y = df["EVADIDO"]

    skf = StratifiedKFold(n_splits = 2)
    skf.get_n_splits(X, y)
    scaler = StandardScaler()
    print(f"StratifiedKFold config: {skf} \n")
    all_predictions = []
    for train_index, test_index in skf.split(X, y):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # return best k element
        all_predictions.append(run_prediction(X_train, X_test, y_train, y_test))
        
        # General model outputs
        model_utils.generate_output(all_predictions)

In [19]:
if __name__ == "__main__":
    main()

StratifiedKFold config: StratifiedKFold(n_splits=2, random_state=None, shuffle=False) 



C:\Users\julio\.conda\envs\nlp_course\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\julio\.conda\envs\nlp_course\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\julio\.conda\envs\nlp_course\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
